BCC Fe Exchange Model
=====================

James K. Glasbrenner

August 11, 2018

## Modules

We load the required classes and functions from the pymatgen library for easy manipulation of crystal structures.

In [1]:
from pymatgen.transformations.standard_transformations import SupercellTransformation

We also import convenient functions from the [neighbormodels](https://github.com/jkglasbrenner/datamaterials-neighbormodels) module that were developed for counting neighbors in a periodic crystal system.

In [2]:
from neighbormodels.structure import from_file
from neighbormodels.neighbors import count_neighbors
from neighbormodels.interactions import build_model

## Crystal structure

At room temperature, [iron has the following crystal structure](#kohlhaas1967_ZAngewPhys):

<table>
  <tr><td>Structure</td><td>Body-centered cubic</td></tr>
  <tr><td>Spacegroup</td><td>Im-3m (229)</td></tr>
  <tr><td>Lattice constants</td><td>a = 2.8665 Å</td></tr>
  <tr>
    <td>Basis</td>
    <td>Fe: [0.00 0.00 0.00]</td>
  </tr>
</table>

A CIF file with these parmaeters that generates a conventional two-atom unit cell for BCC Fe is [available here](data/fe.cif).
We use this file to generate a [pymatgen](http://pymatgen.org/) `Structure` object that will be convenient for building our magnetic exchange model.

In [3]:
fe_cif_filepath = "data/fe.cif"
fe_structure_2atom = from_file(structure_file=fe_cif_filepath)

A quick printout of `fe_structure` confirms that we have the correct structure:

In [4]:
print(fe_structure_2atom)

Full Formula (Fe2)
Reduced Formula: Fe
abc   :   2.866500   2.866500   2.866500
angles:  90.000000  90.000000  90.000000
Sites (2)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Fe    0    0    0
  1  Fe    0.5  0.5  0.5


To represent the f-type, a-type, and g-type magnetic patterns, we will need a four-atom cell instead of a two-atom cell.
We transform the supercell as follows:

In [5]:
rotate_cell_45_degrees = SupercellTransformation(
    scaling_matrix=[[1, 1, 0],
                    [1, -1, 0],
                    [0, 0, 1]],
)
fe_structure_4atom = rotate_cell_45_degrees.apply_transformation(fe_structure_2atom)

As expected, this results in the required 4-atom unit cell:

In [6]:
print(fe_structure_4atom)

Full Formula (Fe4)
Reduced Formula: Fe
abc   :   4.053843   4.053843   2.866500
angles:  90.000000  90.000000  90.000000
Sites (4)
  #  SP      a    b     c
---  ----  ---  ---  ----
  0  Fe    0    0     0
  1  Fe    0.5  0.5  -0
  2  Fe    0.5  0     0.5
  3  Fe    1    0.5   0.5


## Magnetic patterns

Defining a magnetic pattern is straightforward and is achieved by assigning a spin direction to each site in a unit cell.
We adopt the conventional method for building a classical Ising or Heisenberg model by assuming that all spin amplitudes are equal in magnitude on all sites.
This allows us to specify a magnetic pattern by assigning +1 or -1 to each site.
This means unit cells with more sites allow you to define more unique magnetic patterns, which is useful if the magnetic energy landscape is complicated such that modeling these energies requires including more terms.
For this tutorial, the BCC Fe 4-atom cell will suffice.

In [7]:
bcc_4atom_magnetic_patterns = {
    "f-type": [+1, +1, +1, +1],
    "a-type": [+1, +1, -1, -1],
    "g-type": [+1, -1, +1, -1],
}

## Build the model

In [8]:
fe_neighbor_data = count_neighbors(cell_structure=fe_structure_4atom, r=3)

In [9]:
fe_model = build_model(magnetic_patterns=bcc_4atom_magnetic_patterns, neighbor_data=fe_neighbor_data)

## References

1. <a id="kohlhaas1967_ZAngewPhys"></a> R. Kohlhaas, P. Donner, and N. Schmitz-Pranghe, Z. Angew. Phys. 23, 245 (1967)